In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
from model.gpt import GPT
from fastai.text.all import *
from data.unlabeled import download_dataset
from fastai.distributed import *
from learner.LLMLearner import LLMLearner


from data.loader import memmapDL


In [4]:
model = GPT.from_hf('microsoft/Phi-3-mini-4k-instruct', enable_qlora = True)


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.18s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
dataset = "orcamath"
bs = 1
valid_sampler_size = 1000 #how many samples to use for validation. This is only used to check if validation loss is better than best_valid_loss, so that a checkpoint can be saved. Karpathy uses 200 random points
validate_every = 1000 #1000 iterations, each iteration is bs*total_GPUs inputs
qlora = True

train_path, valid_path = rank0_first(lambda: download_dataset(dataset = dataset, encoder = model.tokenizer)) #check if data exists, download only for rank0 GPU. 
train_dl = memmapDL(train_path, bs = bs, block_size=model.block_size, 
                      dtype=model.tokenizer._get_numpy_dtype())
valid_dl = memmapDL(valid_path, bs = bs, block_size=model.block_size, 
                      dtype=model.tokenizer._get_numpy_dtype(), 
                      sample_size = valid_sampler_size)


Setting num_proc from 18 back to 1 for the train split to disable multiprocessing as it only contains one shard.
2024-06-18:11:19:08,114 WARNING  [builder.py:1855] Setting num_proc from 18 back to 1 for the train split to disable multiprocessing as it only contains one shard.
Generating train split: 100%|██████████| 200035/200035 [00:01<00:00, 156801.22 examples/s]


In [7]:
dls = DataLoaders(train_dl, valid_dl)
dls.c = model.vocab_size

In [24]:
learn = LLMLearner(dls, 
                model, 
                opt_func = partial(OptimWrapper, opt=torch.optim.AdamW),
                loss_func=CrossEntropyLossFlat(), 
                metrics=[accuracy, Perplexity()],
                ).to_bf16()
learn.path = Path('scripts/checkpoints/') #local path to save/load checkpoints
learn.model_dir = 'gpt'

In [25]:
learn.load(Path('Phi-3-mini-25.2M'))

In [26]:
tasks = [
    'mmlu_abstract_algebra', 
         'mmlu_college_mathematics',
         'mmlu_elementary_mathematics',
         'mmlu_high_school_mathematics',
         'mmlu_formal_logic',
         'arithmetic',
        #  'minerva_math',
        #  'hendrycks_math',
         'mathqa', 
        #  'gsm8k',
         ]

In [27]:
learn.model.evaluate(tasks = tasks)

2024-06-18:11:26:10,185 INFO     [evaluator.py:141] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234
2024-06-18:11:26:10,186 INFO     [evaluator.py:192] Using pre-initialized model


running eval. This may take a few minutes just to setup...


KeyboardInterrupt: 

: 